In [1]:
# Import library

import pandas as pd
import requests

In [2]:
# Download dataset

response = requests.get("https://raw.githubusercontent.com/yinghaoz1/tmdb-movie-dataset-analysis/master/tmdb-movies.csv")
with open("tmdb-movies.csv", "wb") as wf:
    wf.write(response.content)

In [3]:
#Load dataset

df = pd.read_csv("tmdb-movies.csv")

In [4]:
# Process Date type
def format_date(row):
    try:
        date = row['release_date']
        parts = date.split('/')
        month = parts[0].zfill(2)
        day = parts[1].zfill(2)
        year = str(row['release_year'])
        return f"{year}/{month}/{day}"
    except:
        return None

df['release_date'] = df.apply(format_date, axis=1)

In [5]:
# 1. Sort movies by release date in descending order and save to a new file
sorted_movies_by_date = df.sort_values(by='release_date', ascending=False)
sorted_movies_by_date.to_csv('output/sorted_movies_by_date.csv', index=False)

In [6]:
# 2. Filter out movies with an average voting above 7.5 and save to a new file
above_75_rate_movies = df[df['vote_average'] > 7.5]
above_75_rate_movies.to_csv('output/above_75_rate_movies.csv', index=False)

print(f"The number of above 7.5 rate movies: {len(above_75_rate_movies)}")

The number of above 7.5 rate movies: 350


In [7]:
# Ensure 'revenue' and 'revenue_adj' are numeric
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['revenue_adj'] = pd.to_numeric(df['revenue_adj'], errors='coerce')

# 3.1 By 'revenue' field
max_revenue = df[df['revenue'] == df['revenue'].max()]
min_revenue = df[df['revenue'] == df['revenue'].min()]
max_revenue.to_csv('output/max_revenue.csv', index=False)
min_revenue.to_csv('output/min_revenue.csv', index=False)

print(f"The number of movies with max revenue: {len(max_revenue)}")
print(f"The number of movies with min revenue: {len(min_revenue)}")

# 3.2 By 'revenue_adj' field
max_revenue_adj = df[df['revenue_adj'] == df['revenue_adj'].max()]
min_revenue_adj = df[df['revenue_adj'] == df['revenue_adj'].min()]
max_revenue_adj.to_csv('output/max_revenue_adj.csv', index=False)
min_revenue_adj.to_csv('output/min_revenue_adj.csv', index=False)

print(f"The number of movies with max revenue adj: {len(max_revenue_adj)}")
print(f"The number of movies with min revenue adj: {len(min_revenue_adj)}")


The number of movies with max revenue: 1
The number of movies with min revenue: 6016
The number of movies with max revenue adj: 1
The number of movies with min revenue adj: 6016


In [8]:
# 4. Calculate total revenue of all movies
# 4.1 Total revenue
sum_revenue = df['revenue'].sum()

# 4.2 Total revenue_adj
sum_revenue_adj = df['revenue_adj'].sum()

with open ('output/single_output.txt', 'a', encoding='utf-8') as wf:
    wf.write(f"The sum of revenue by revenue field: {sum_revenue:.10f} dollars\n")
    wf.write(f"The sum of revenue by revenue_adj field: {sum_revenue_adj:.10f} dollars\n")

In [9]:
# 5. Top 10 highest profits
# 5.1 By revenue and budget
df['profit'] = df['revenue'] - df['budget']
top_profit = df.sort_values(by='profit', ascending=False).head(10)
top_profit.to_csv('output/top_profit.csv', index=False)

# 5.2 By revenue_adj and budget_adj
df['profit_adj'] = df['revenue_adj'] - df['budget_adj']
top_profit_adj = df.sort_values(by='profit_adj', ascending=False).head(10)
top_profit_adj.to_csv('output/top_profit_adj.csv', index=False)

In [10]:
# 6. Which director has the most films and which actor has the most films
# Director
da = df[df['director'].notna()]  # Get rows with not null in director column
all_directors = da['director'].str.split('|').explode().str.strip()
all_directors = all_directors[all_directors != ''] # after splitting, there exist empty

director_count = all_directors.value_counts()
most_directed = director_count.idxmax()
num_movies_directed = director_count.max()

# Actor or Actress
dv = df[df['cast'].notna()] # Get rows with not null in cast column
all_actors = dv['cast'].str.split('|').explode().str.strip()
all_actors = all_actors[all_actors != ''] # after splitting, there exist empty


actor_counts = all_actors.value_counts()
most_common_actor = actor_counts.idxmax()
num_movies = actor_counts.max()

with open ('output/single_output.txt', 'a', encoding='utf-8') as wf:
    wf.write(f"The director has the most movies is: {most_directed} ({num_movies_directed} movies)\n")
    wf.write(f"The actor or actress has the most movies is: {most_common_actor} ({num_movies} movies)\n")


In [11]:
# 7. Count the number of movies by genre.
# For example, how many movies are in the Action genre, how many are in the Family genre, ….
dg = df[df['genres'].notna()] # Get rows with not null in genres column
all_genres = dg['genres'].str.split('|').explode().str.strip()
all_genres = all_genres[all_genres != ''] # after splitting, there exist empty

genres_count = all_genres.value_counts()

with open('output/statistic_movie_genres.txt', 'w', encoding='utf-8') as wf:
    for genre, count in genres_count.items():
        wf.write(f"{genre}: {count}\n")